In [1]:
import os
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from configs import kafka_config, COMMON_ALERT_TOPIC_NAME

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell'

spark = SparkSession.builder \
    .appName("KafkaReadStreamPrinter") \
    .getOrCreate()

24/11/26 16:23:20 WARN Utils: Your hostname, msolonin-ThinkPad resolves to a loopback address: 127.0.1.1; using 192.168.0.214 instead (on interface wlp0s20f3)
24/11/26 16:23:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/msolonin/PycharmProjects/GoIT/Data-Engineering/.venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/msolonin/.ivy2/cache
The jars for the packages stored in: /home/msolonin/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4b858409-d961-4651-891b-28c002884190;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.spark#spark-sql-kafka

In [3]:
kafka_stream_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_config['bootstrap_servers'][0]) \
    .option("subscribe", COMMON_ALERT_TOPIC_NAME) \
    .option("kafka.security.protocol", kafka_config['security_protocol']) \
    .option("kafka.sasl.mechanism", kafka_config['sasl_mechanism']) \
    .option("kafka.sasl.jaas.config",
            "org.apache.kafka.common.security.plain.PlainLoginModule required username='admin' password='VawEzo1ikLtrA8Ug8THa';") \
    .option("startingOffsets", "latest") \
    .load()


In [4]:
messages_df = kafka_stream_df.select(
    col("key").cast("string"),
    col("value").cast("string")
)

query = messages_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

24/11/26 16:23:27 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-37eef03f-c283-495a-9866-56e8674183dd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/11/26 16:23:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/11/26 16:23:28 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
+---+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|4759d968-025b-4d5...|{"window":{"start...|
+--------------------+--------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|0f3cba08-beda-4ca...|{"window":{"start...|
+--------------------+--------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|c4a2ec53-fa10-485...|{"window":{"start...|
+--------------------+--------------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|a37ec9a1-c3ca-4d9...|{"window":{"start...|
+--------------------+--------------------+



-------------------------------------------
Batch: 5
-------------------------------------------
+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|4aa3539f-b185-4c8...|{"window":{"start...|
+--------------------+--------------------+

-------------------------------------------
Batch: 6
-------------------------------------------
+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|f5a7a79e-0c1d-45e...|{"window":{"start...|
+--------------------+--------------------+



-------------------------------------------
Batch: 7
-------------------------------------------
+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|ccbe9fd0-5430-47d...|{"window":{"start...|
|73c21e0f-1048-4f8...|{"window":{"start...|
+--------------------+--------------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/msolonin/PycharmProjects/GoIT/Data-Engineering/.venv/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/msolonin/PycharmProjects/GoIT/Data-Engineering/.venv/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 